# Navigation using Waypoints


In [1]:
import glob
import os
import sys
import random
import time
import numpy as np
import math
import glob
import os
import sys

from enum import Enum
import networkx as nx
from dataclasses import dataclass
from queue import Queue
from collections import deque
from queue import Queue

import tensorflow as tf

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

print(tf.__version__)

2.5.0


In [2]:
# actor_list = []   #store all spawned actor objects of the simulation in this list

# client = carla.Client("localhost", 2000)         #connecting to the server
# client.set_timeout(10.0)                         #show timeout error if it takes longer than 2 seconds to connect to server

# world = client.get_world()
# wmap = world.get_map()

In [3]:
class GlobalRoutePlannerDAO(object):
    """
    This class is the data access layer for fetching data
    from the carla server instance for GlobalRoutePlanner
    """

    def __init__(self, wmap, sampling_resolution=1.0):
        """get_topology
        Constructor
        wmap    :   carl world map object
        """
        self._sampling_resolution = sampling_resolution
        self._wmap = wmap

    def get_topology(self):
        """
        Accessor for topology.
        This function retrieves topology from the server as a list of
        road segments as pairs of waypoint objects, and processes the
        topology into a list of dictionary objects.
        return: list of dictionary objects with the following attributes
                entry   -   waypoint of entry point of road segment
                entryxyz-   (x,y,z) of entry point of road segment
                exit    -   waypoint of exit point of road segment
                exitxyz -   (x,y,z) of exit point of road segment
                path    -   list of waypoints separated by 1m from entry
                            to exit
        """
        topology = []
        # Retrieving waypoints to construct a detailed topology
        for segment in self._wmap.get_topology():
            wp1, wp2 = segment[0], segment[1]
            l1, l2 = wp1.transform.location, wp2.transform.location
            # Rounding off to avoid floating point imprecision
            x1, y1, z1, x2, y2, z2 = np.round([l1.x, l1.y, l1.z, l2.x, l2.y, l2.z], 0)
            wp1.transform.location, wp2.transform.location = l1, l2
            seg_dict = dict()
            seg_dict['entry'], seg_dict['exit'] = wp1, wp2
            seg_dict['entryxyz'], seg_dict['exitxyz'] = (x1, y1, z1), (x2, y2, z2)
            seg_dict['path'] = []
            endloc = wp2.transform.location
            if wp1.transform.location.distance(endloc) > self._sampling_resolution:
                w = wp1.next(self._sampling_resolution)[0]      #w = np.random.choice(wp1.next(self._sampling_resolution))
                while w.transform.location.distance(endloc) > self._sampling_resolution:
                    seg_dict['path'].append(w)
                    w = w.next(self._sampling_resolution)[0]    #w = np.random.choice(wp1.next(self._sampling_resolution))  
            else:
                seg_dict['path'].append(wp1.next(self._sampling_resolution/2.0)[0])
            topology.append(seg_dict)
        return topology

    def get_waypoint(self, location):
        """
        The method returns waypoint at given location
        """
        waypoint = self._wmap.get_waypoint(location)
        return waypoint

    def get_resolution(self):
        """ Accessor for self._sampling_resolution """
        return self._sampling_resolution

In [4]:
# GRP = GlobalRoutePlannerDAO(wmap)
# topology = GRP.get_topology()
# print(len(topology))
# topology[0]

In [5]:
def draw_waypoints(world, waypoints, z=0.5):
    """
    Draw a list of waypoints at a certain height given in z.
    :param world: carla.world object
    :param waypoints: list or iterable container with the waypoints to draw
    :param z: height in meters
    :return:
    """
    for w in waypoints:
        t = w.transform
        begin = t.location + carla.Location(z=z)
        angle = math.radians(t.rotation.yaw)
        end = begin + carla.Location(x=math.cos(angle), y=math.sin(angle))
        world.debug.draw_arrow(begin, end, arrow_size=0.3, life_time=10.0)
        
def get_speed(vehicle):
    """
    Compute speed of a vehicle in Kmh
    :param vehicle: the vehicle for which speed is calculated
    :return: speed as a float in Kmh
    """
    vel = vehicle.get_velocity()
    return 3.6 * math.sqrt(vel.x ** 2 + vel.y ** 2 + vel.z ** 2)

def is_within_distance_ahead(target_location, current_location, orientation, max_distance):
    """
    Check if a target object is within a certain distance in front of a reference object.
    :param target_location: location of the target object
    :param current_location: location of the reference object
    :param orientation: orientation of the reference object
    :param max_distance: maximum allowed distance
    :return: True if target object is within max_distance ahead of the reference object
    """
    target_vector = np.array([target_location.x - current_location.x, target_location.y - current_location.y])
    norm_target = np.linalg.norm(target_vector)

    # If the vector is too short, we can simply stop here
    if norm_target < 0.001:
        return True

    if norm_target > max_distance:
        return False

    forward_vector = np.array(
        [math.cos(math.radians(orientation)), math.sin(math.radians(orientation))])
    d_angle = math.degrees(math.acos(np.dot(forward_vector, target_vector) / norm_target))

    return d_angle < 90.0

def compute_magnitude_angle(target_location, current_location, orientation):
    """
    Compute relative angle and distance between a target_location and a current_location
    :param target_location: location of the target object
    :param current_location: location of the reference object
    :param orientation: orientation of the reference object
    :return: a tuple composed by the distance to the object and the angle between both objects
    """
    target_vector = np.array([target_location.x - current_location.x, target_location.y - current_location.y])
    norm_target = np.linalg.norm(target_vector)

    forward_vector = np.array([math.cos(math.radians(orientation)), math.sin(math.radians(orientation))])
    d_angle = math.degrees(math.acos(np.dot(forward_vector, target_vector) / norm_target))

    return (norm_target, d_angle)

def distance_vehicle(waypoint, vehicle_transform):
    """
    Compute euclidean distance between vehicle and given waypoint
    :param waypoint: waypoint from which distance of vehicle is to be calculated
    :param vehicle_transform: transform object of the reference vehicle
    """
    loc = vehicle_transform.location
    dx = waypoint.transform.location.x - loc.x
    dy = waypoint.transform.location.y - loc.y

    return math.sqrt(dx * dx + dy * dy)

def vector(location_1, location_2):
    """
    Returns the unit vector from location_1 to location_2
    location_1, location_2:   carla.Location objects
    """
    x = location_2.x - location_1.x
    y = location_2.y - location_1.y
    z = location_2.z - location_1.z
    norm = np.linalg.norm([x, y, z]) + np.finfo(float).eps

    return [x / norm, y / norm, z / norm]



In [6]:
class RoadOption(Enum):
    """
    RoadOption represents the possible topological configurations when moving from a segment of lane to other.
    """
    VOID = -1
    LEFT = 1
    RIGHT = 2
    STRAIGHT = 3
    LANEFOLLOW = 4
    CHANGELANELEFT = 5
    CHANGELANERIGHT = 6

In [7]:
class GlobalRoutePlanner(object):
    """
    This class provides a very high level route plan.
    Instantiate the class by passing a reference to 
    A GlobalRoutePlannerDAO object
    """
    
    def __init__(self, dao):
        """
        Constructor
        """
        self._dao = dao
        self._topology = None
        self._graph = None
        self._id_map = None
        self._road_id_to_edge = None
        self._intersection_end_node = -1
        self._previous_decision = RoadOption.VOID
        
    def setup(self):
        """Performs initial server data lookup for detailed topology
        and builds graph representation of the world map
        """
        self._topology = self._dao.get_topology()
        self._graph, self._id_map, self._road_id_to_edge = self._build_graph()
        self._find_loose_ends()
        self._lane_change_link()
        
    def _build_graph(self):
        """
        This function builds a network graph representation of topology.
        The topology is read from self._topology.
        graph node properties:
            vertex - (x, y, z) position in world map
        graph edge properties:
            entry_vector - unit vector along tangent to entry point
            exit_vector - unit vector along tangent to exit point
            net_vector - unit vector of the chord from entry to exit
            intersection - boolean indicating if the edge belongs to an 
                           intersection
        return: graph -> netwrokx graph representing the world map,
                id_map -> mapping from (x, y, z) to node_id
                road_id_to_edge -> map from road id to edge in the graph
        """
        graph = nx.DiGraph()
        id_map = dict()    # Map with structure {(x, y, z): node_id, ...}
        road_id_to_edge = dict()   # Map with structure {road_id: {lane_id: edge, ...}, ...}
        
        """
        id_map[(x, y, z)] -> node_id
        road_id_to_edge[road_id][section_id][lane_id] -> (node_id for entry_xyz, node_id for exit_xyz) / (node_id for exit_xyz, -1*loose_end_count)
        """
        
        for segment in self._topology:
            
            entry_xyz, exit_xyz = segment['entryxyz'], segment['exitxyz']
            path = segment['path']
            entry_wp, exit_wp = segment['entry'], segment['exit']
            intersection = entry_wp.is_junction
            road_id, section_id, lane_id = entry_wp.road_id, entry_wp.section_id, entry_wp.lane_id
            
            for vertex in [entry_xyz, exit_xyz]:
                #Adding unique nodes and populating id_map
                if vertex not in id_map:
                    new_id = len(id_map)
                    id_map[vertex] = new_id
                    graph.add_node(new_id, vertex=vertex)
            n1 = id_map[entry_xyz]
            n2 = id_map[exit_xyz]
            if road_id not in road_id_to_edge:
                road_id_to_edge[road_id] = dict()
            if section_id not in road_id_to_edge[road_id]:
                road_id_to_edge[road_id][section_id] = dict()
            road_id_to_edge[road_id][section_id][lane_id] = (n1, n2)
            
            entry_carla_vector = entry_wp.transform.rotation.get_forward_vector()   # vector pointing forward from entry waypoint of road
            exit_carla_vector = exit_wp.transform.rotation.get_forward_vector()
            #Adding edge with attributes
            graph.add_edge(n1, n2,
                          length=len(path)+1,
                          path=path,
                          entry_waypoint=entry_wp,
                          exit_waypoint=exit_wp,
                          entry_vector=np.array([entry_carla_vector.x, entry_carla_vector.y, entry_carla_vector.z]),
                          exit_vector=np.array([exit_carla_vector.x, exit_carla_vector.y, exit_carla_vector.z]),
                          net_vector=vector(entry_wp.transform.location, exit_wp.transform.location),
                          intersection=intersection,
                          type=RoadOption.LANEFOLLOW)
            
        return graph, id_map, road_id_to_edge
    
    def _find_loose_ends(self):
        """
        This method finds road segments that have an unconnected end and
        adds them to the internal graph representation.
        Some road exit points are entry points for other roads, but not all exit points.
        These exit points have to be seperately added to our dict. (previously only entry points were added)
        """
        count_loose_ends = 0
        hop_resolution = self._dao.get_resolution()
        for segment in self._topology:
            end_wp = segment['exit']
            exit_xyz = segment['exitxyz']
            road_id, section_id, lane_id = end_wp.road_id, end_wp.section_id, end_wp.lane_id
            if road_id in self._road_id_to_edge and section_id in self._road_id_to_edge[road_id] and lane_id in self._road_id_to_edge[road_id][section_id]:
                pass
            else:
                count_loose_ends += 1
                if road_id not in self.road_id_to_edge:
                    self._road_id_to_edge[road_id] = dict()
                if section_id not in self._road_id_to_edge[road_id]:
                    self._road_id_to_edge[road_id][section_id] = dict()
                n1 = self._id_map[exit_xyz]
                n2 = -1*count_loose_ends
                self._road_id_to_edge[road_id][section_id][lane_id] = (n1, n2)
                next_wp = end_wp.next(hop_resolution)
                path = []
                while next_wp is not None and nxt_wp and next_wp[0].road_id == road_id and next_wp[0].section_id == section_id and next_wp[0].lane_id == lane_id:
                    path.append(next_wp[0])
                    next_wp = next_wp[0].next(hop_resolution)
                if path:
                    n2_xyz = (path[-1].transform.location.x,
                              path[-1].transform.location.y,
                              path[-1].transform.location.z)
                    self._graph.add_node(n2, vertex=n2_xyz)
                    self._graph.add_edge(
                        n1, n2,
                        length=len(path)+1, 
                        path=path,
                        entry_waypoint=end_wp, 
                        exit_waypoint=path[-1],
                        entry_vector=None, 
                        exit_vector=None, 
                        net_vector=None,
                        intersection=end_wp.is_intersection, 
                        type=RoadOption.LANEFOLLOW)
                    
    def _localize(self, location):
        """
        This function finds the road segment closest to given location
        location  : carla.Location to be lcoalized in the graph
        return    : pair node ids representing an edge in the graph
        """
        waypoint = self._dao.get_waypoint(location)
        edge = None
        try:
            edge = self._road_id_to_edge[waypoint.road_id][waypoint.section_id][waypoint.lane_id]
        except:
            print(
                "Failed to localize! : ",
                "Road id : ", waypoint.road_id,
                "Section id : ", waypoint.section_id,
                "Lane id : ", waypoint.lane_id,
                "Location : ", waypoint.transform.location.x,
                waypoint.transform.location.y)
        return edge
    
    def _lane_change_link(self):
        """
        This method places zero cost links in the topology graph
        representing availability of lane changes.
        The edge property change_waypoint is added to edges(roads) which allow for left or right lane change
        """
        
        for segment in self._topology:
            left_found, right_found = False, False
            
            for waypoint in segment['path']:
                if not segment['entry'].is_junction:
                    next_waypoint, next_road_option, next_segment = None, None, None
                    
                    if bool(waypoint.lane_change & carla.LaneChange.Right) and not right_found:
                        next_waypoint = waypoint.get_right_lane()
                        if next_waypoint is not None and next_waypoint.lane_type == carla.LaneType.Driving and waypoint.road_id == next_waypoint.road_id:
                            next_road_option = RoadOption.CHANGELANERIGHT
                            next_segment = self._localize(next_waypoint.transform.location)
                            if next_segment is not None:
                                self._graph.add_edge(self._id_map[segment['entryxyz']], 
                                                     next_segment[0], 
                                                     entry_waypoint=segment['entry'],
                                                     exit_waypoint=self._graph.edges[next_segment[0], next_segment[1]]['entry_waypoint'],
                                                     path=[], 
                                                     length=0, 
                                                     type=next_road_option, 
                                                     change_waypoint = waypoint)
                                right_found = True
                                
                    if bool(waypoint.lane_change & carla.LaneChange.Left) and not left_found:
                        next_waypoint = waypoint.get_left_lane()
                        if next_waypoint is not None and next_waypoint.lane_type == carla.LaneType.Driving and \
                            waypoint.road_id == next_waypoint.road_id:
                            next_road_option = RoadOption.CHANGELANELEFT
                            next_segment = self._localize(next_waypoint.transform.location)
                            if next_segment is not None:
                                self._graph.add_edge(self._id_map[segment['entryxyz']], 
                                                     next_segment[0], 
                                                     entry_waypoint=segment['entry'],
                                                     exit_waypoint=self._graph.edges[next_segment[0], next_segment[1]]['entry_waypoint'],
                                                     path=[], 
                                                     length=0, 
                                                     type=next_road_option, 
                                                     change_waypoint = waypoint)
                                left_found = True

                if left_found and right_found:
                    break
                    
    def _distance_heuristic(self, n1, n2):
        """
        Distance heuristic calculator for path searching in self.graph
        """
        l1 = np.array(self._graph.nodes[n1]['vertex'])
        l2 = np.array(self._graph.nodes[n2]['vertex'])
        
        return np.linalg.norm(l1-l2)      #distance between n1 and n2
    
    def _path_search(self, origin, destination):
        """
        This function finds the shortest path connecting origin and destination
        using A* search with distance heuristic.
        origin     : carla.Location object of start position
        destination: carla.Location object of end position
        return     : path as list of node ids (as int) of the graph self._graph
        connecting origin and destination
        """
        
        start, end = self._localize(origin), self._localize(destination)
        
        route, end = self._localize(origin), self._localize(destination)
        
        route = nx.astar_path(self._graph, 
                             source=start[0], 
                             target=end[0],
                             heuristic=self._distance_heuristic,
                             weight='length')
        route.append(end[1])
        
        return route   #list of node_ids
    
    def _successive_last_intersection_edge(self, index, route):
        """
        This method returns the last successive intersection edge
        from a starting index on the route.
        
        This helps moving past tiny intersection edges to calculate
        proper turn decisions.
        
        index: a node from the graph
        """
        
        last_intersection_edge = None
        last_node = None
        
        for node1, node2 in [(route[i], route[i+1]) for i in range(index, len(route)-1)]:
            candidate_edge = self._graph.edges[node1, node2]
            if node1 == route[index]:
                last_intersection_edge = candidate_edge
            if candidate_edge['type'] == RoadOption.LANEFOLLOW and candidate_edge['intersection']:
                last_intersection_edge = candidate_edge
                last_node = node2
            else:
                break
                
        return last_node, last_intersection_edge
    
    def _turn_decision(self, index, route, threshold=math.radians(5)):    #math.radians(5) returns 5 degrees in radians\
        """
        This method returns the turn decision (RoadOption) for pair of edges
        around current index of route list
        index     : a node from the graph
        threshold : in degrees
        return    : RoadOption type 
        """
        
        decision = None
        previous_node = route[index-1]
        current_node = route[index]
        next_node = route[index+1]
        next_edge = self._graph.edges[current_node, next_node]
        if index > 0:
            if self._previous_decision != RoadOption.VOID and \
                self._intersection_end_node > 0 and \
                    self._intersection_end_node != previous_node and \
                        next_edge['type'] == RoadOption.LANEFOLLOW and \
                            next_edge['intersection']:
                decision = self._previous_decision
            else:
                self._intersection_end_node = -1
                current_edge = self._graph.edges[previous_node, current_node]
                calculate_turn = current_edge['type'].value == RoadOption.LANEFOLLOW.value and \
                    not current_edge['intersection'] and \
                        next_edge['type'].value == RoadOption.LANEFOLLOW.value and \
                            next_edge['intersection']
                if calculate_turn:
                    last_node, tail_edge = self._successive_last_intersection_edge(index, route)
                    self._intersection_end_node = last_node
                    if tail_edge is not None:
                        next_edge = tail_edge
                    cv, nv = current_edge['exit_vector'], next_edge['net_vector']
                    cross_list = []
                    for neighbor in self._graph.successors(current_node):
                        select_edge = self._graph.edges[current_node, neighbor]
                        if select_edge['type'].value == RoadOption.LANEFOLLOW.value:
                            if neighbor != route[index+1]:
                                sv = select_edge['net_vector']
                                cross_list.append(np.cross(cv, sv)[2])
                    next_cross = np.cross(cv, nv)[2]
                    deviation = math.acos(np.clip(
                        np.dot(cv, nv)/(np.linalg.norm(cv)*np.linalg.norm(nv)), -1.0, 1.0))
                    if not cross_list:
                        cross_list.append(0)
                    if deviation < threshold:
                        decision = RoadOption.STRAIGHT
                    elif cross_list and next_cross < min(cross_list):
                        decision = RoadOption.LEFT
                    elif cross_list and next_cross > max(cross_list):
                        decision = RoadOption.RIGHT
                    elif next_cross < 0:
                        decision = RoadOption.LEFT
                    elif next_cross > 0:
                        decision = RoadOption.RIGHT
                else:
                    decision = next_edge['type']
        else:
            decision = next_edge['type']
        self._previous_decision = decision

        return decision
    
    def abstract_route_plan(self, origin, destination):
        """
        The following function generates the route plan based on
        origin      : carla.Location object of the route's start position
        destination : carla.Location object of the route's end position
        return      : list of turn by turn navigation decisions as
        agents.navigation.local_planner.RoadOption elements
        Possible values are STRAIGHT, LEFT, RIGHT, LANEFOLLOW, VOID
        CHANGELANELEFT, CHANGELANERIGHT
        """

        route = self._path_search(origin, destination)
        plan = []

        for i in range(len(route) - 1):
            road_option = self._turn_decision(i, route)
            plan.append(road_option)

        return plan
    
    def _find_closest_in_list(self, current_waypoint, waypoint_list):
        min_distance = float('inf')
        closest_index = -1
        for i, waypoint in enumerate(waypoint_list):
            distance = waypoint.transform.location.distance(
                current_waypoint.transform.location)
            if distance < min_distance:
                min_distance = distance
                closest_index = i

        return closest_index

    def trace_route(self, origin, destination):
        """
        This method returns list of (carla.Waypoint, RoadOption)
        from origin (carla.Location) to destination (carla.Location)
        """

        route_trace = []
        route = self._path_search(origin, destination)
        current_waypoint = self._dao.get_waypoint(origin)
        destination_waypoint = self._dao.get_waypoint(destination)
        resolution = self._dao.get_resolution()

        for i in range(len(route) - 1):
            road_option = self._turn_decision(i, route)
            edge = self._graph.edges[route[i], route[i+1]]
            path = []

            if edge['type'].value != RoadOption.LANEFOLLOW.value and \
                edge['type'].value != RoadOption.VOID.value:
                route_trace.append((current_waypoint, road_option))
                exit_wp = edge['exit_waypoint']
                n1, n2 = self._road_id_to_edge[exit_wp.road_id][exit_wp.section_id][exit_wp.lane_id]
                next_edge = self._graph.edges[n1, n2]
                if next_edge['path']:
                    closest_index = self._find_closest_in_list(current_waypoint, next_edge['path'])
                    closest_index = min(len(next_edge['path'])-1, closest_index+5)
                    current_waypoint = next_edge['path'][closest_index]
                else:
                    current_waypoint = next_edge['exit_waypoint']
                route_trace.append((current_waypoint, road_option))

            else:
                path = path + [edge['entry_waypoint']] + edge['path'] + [edge['exit_waypoint']]
                closest_index = self._find_closest_in_list(current_waypoint, path)
                for waypoint in path[closest_index:]:
                    current_waypoint = waypoint
                    route_trace.append((current_waypoint, road_option))
                    if len(route)-i <= 2 and \
                        waypoint.transform.location.distance(destination) < 2*resolution:
                        break
                    elif len(route)-i <= 2 and \
                        current_waypoint.road_id == destination_waypoint.road_id and \
                            current_waypoint.section_id == destination_waypoint.section_id and \
                                current_waypoint.lane_id == destination_waypoint.lane_id:
                        destination_index = self._find_closest_in_list(destination_waypoint, path)
                        if closest_index > destination_index:
                            break

        return route_trace
            

        

In [8]:
class VehiclePIDController():
    """
    VehiclePIDController is the combination of two PID controllers (lateral and longitudinal) to perform the
    low level control of a vehicle from client side
    """

    def __init__(self, vehicle, args_lateral=None, args_longitudinal=None):
        """
        :param vehicle: actor to apply to local planner logic onto
        :param args_lateral: dictionary of arguments to set the lateral PID controller using the following semantics:
                             K_P -- Proportional term
                             K_D -- Differential term
                             K_I -- Integral term
                             dt  -- time difference
        :param args_longitudinal: dictionary of arguments to set the longitudinal PID controller using the following
        semantics:
                             K_P -- Proportional term
                             K_D -- Differential term
                             K_I -- Integral term
                             dt  -- time difference
        """
        if not args_lateral:
            args_lateral = {'K_P': 1.0, 'K_D': 0.0, 'K_I': 0.0, 'dt':0.03}
        if not args_longitudinal:
            args_longitudinal = {'K_P': 1.0, 'K_D': 0.0, 'K_I': 0.0, 'dt':0.03}

        self._vehicle = vehicle
        self._world = self._vehicle.get_world()
        self._lon_controller = PIDLongitudinalController(self._vehicle, **args_longitudinal)
        self._lat_controller = PIDLateralController(self._vehicle, **args_lateral)

    def run_step(self, target_speed, waypoint):
        """
        Execute one step of control invoking both lateral and longitudinal PID controllers to reach a target waypoint
        at a given target_speed.
        :param target_speed: desired vehicle speed
        :param waypoint: target location encoded as a waypoint
        :return: distance (in meters) to the waypoint
        """
        throttle = self._lon_controller.run_step(target_speed)
        steering = self._lat_controller.run_step(waypoint)

        control = carla.VehicleControl()
        control.steer = steering
        control.throttle = throttle
        control.brake = 0.0
        control.hand_brake = False
        control.manual_gear_shift = False

        return control


class PIDLongitudinalController():
    """
    PIDLongitudinalController implements longitudinal control using a PID.
    """

    def __init__(self, vehicle, K_P=1.0, K_D=0.0, K_I=0.0, dt=0.03):
        """
        :param vehicle: actor to apply to local planner logic onto
        :param K_P: Proportional term
        :param K_D: Differential term
        :param K_I: Integral term
        :param dt: time differential in seconds
        """
        self._vehicle = vehicle
        self._K_P = K_P
        self._K_D = K_D
        self._K_I = K_I
        self._dt = dt
        self._e_buffer = deque(maxlen=30)

    def run_step(self, target_speed, debug=False):
        """
        Execute one step of longitudinal control to reach a given target speed.
        :param target_speed: target speed in Km/h
        :return: throttle control in the range [0, 1]
        """
        current_speed = get_speed(self._vehicle)

        if debug:
            print('Current speed = {}'.format(current_speed))

        return self._pid_control(target_speed, current_speed)

    def _pid_control(self, target_speed, current_speed):
        """
        Estimate the throttle of the vehicle based on the PID equations
        :param target_speed:  target speed in Km/h
        :param current_speed: current speed of the vehicle in Km/h
        :return: throttle control in the range [0, 1]
        """
        _e = (target_speed - current_speed)
        self._e_buffer.append(_e)

        if len(self._e_buffer) >= 2:
            _de = (self._e_buffer[-1] - self._e_buffer[-2]) / self._dt
            _ie = sum(self._e_buffer) * self._dt
        else:
            _de = 0.0
            _ie = 0.0

        return np.clip((self._K_P * _e) + (self._K_D * _de / self._dt) + (self._K_I * _ie * self._dt), 0.0, 1.0)

        # cl = 1
        # sl = cl * current_speed - target_speed
        # a0 = 1
        # throttle = np.clip(-a0 * np.sign(sl), 0.0, 1.0)
        # print('throttle %0.4f'%throttle)
        # return throttle


class PIDLateralController():
    """
    PIDLateralController implements lateral control using a PID.
    """

    def __init__(self, vehicle, K_P=1.0, K_D=0.0, K_I=0.0, dt=0.03):
        """
        :param vehicle: actor to apply to local planner logic onto
        :param K_P: Proportional term
        :param K_D: Differential term
        :param K_I: Integral term
        :param dt: time differential in seconds
        """
        self._vehicle = vehicle
        self._K_P = K_P
        self._K_D = K_D
        self._K_I = K_I
        self._dt = dt
        self._e_buffer = deque(maxlen=10)

        self.delta = np.array([0,0])

    def run_step(self, waypoint):
        """
        Execute one step of lateral control to steer the vehicle towards a certain waypoint.
        :param waypoint: target waypoint
        :return: steering control in the range [-1, 1] where:
            -1 represent maximum steering to left
            +1 maximum steering to right
        """
        return self._pid_control(waypoint, self._vehicle.get_transform())

    def _pid_control(self, waypoint, vehicle_transform):
        """
        Estimate the steering angle of the vehicle based on the PID equations
        :param waypoint: target waypoint
        :param vehicle_transform: current transform of the vehicle
        :return: steering control in the range [-1, 1]
        """
        d = self.delta[-1]
        v_begin = vehicle_transform.location
        v_end = v_begin + carla.Location(x=math.cos(math.radians(vehicle_transform.rotation.yaw)),
                                         y=math.sin(math.radians(vehicle_transform.rotation.yaw)))

        v_vec = np.array([v_end.x - v_begin.x, v_end.y - v_begin.y, 0.0])
        w_vec = np.array([waypoint.transform.location.x -
                          v_begin.x, waypoint.transform.location.y -
                          v_begin.y, 0.0])
        _dot = math.acos(np.clip(np.dot(w_vec, v_vec) /
                                 (np.linalg.norm(w_vec) * np.linalg.norm(v_vec)), -1.0, 1.0))

        _cross = np.cross(v_vec, w_vec)
        if _cross[2] < 0:
            _dot *= -1.0

        self._e_buffer.append(_dot)
        if len(self._e_buffer) >= 2:
            _de = (self._e_buffer[-1] - self._e_buffer[-2]) / self._dt
            _ie = sum(self._e_buffer) * self._dt
        else:
            _de = 0.0
            _ie = 0.0

        return np.clip((self._K_P * _dot) + (self._K_D * _de /
                                             self._dt) + (self._K_I * _ie * self._dt), -1.0, 1.0)

        # cr = 1
        # crr = 10
        # sr = cr * _ie + crr*_dot
        # print('ie %0.4f'%_ie)
        # print('de %0.4f'%_de)
        # print('dot %0.4f'%_dot)
        # print( 'sr %0.4f'%sr)
        # u0 = 0.01
        # u = -u0 * np.sign(sr)
        # # new_delta = d + u * self._dt
        # new_delta = u
        # print('steering %.4f'%new_delta)
        #
        # self.delta = np.append(self.delta,new_delta)
        # return np.clip(new_delta, -1.0, 1.0)


In [9]:
class ModifiedLocalPlanner(object):
    """
    LocalPlanner implements the basic behavior of following a trajectory of waypoints that is generated on-the-fly.
    The low-level motion of the vehicle is computed by using two PID controllers, one is used for the lateral control
    and the other for the longitudinal control (cruise speed).
    When multiple paths are available (intersections) this local planner makes a random choice.
    """

    # minimum distance to target waypoint as a percentage (e.g. within 90% of
    # total distance)
    MIN_DISTANCE_PERCENTAGE = 0.9

    def __init__(self, vehicle, opt_dict=None):
        """
        :param vehicle: actor to apply to local planner logic onto
        :param opt_dict: dictionary of arguments with the following semantics:
            dt -- time difference between physics control in seconds. This is typically fixed from server side
                  using the arguments -benchmark -fps=F . In this case dt = 1/F
            target_speed -- desired cruise speed in Km/h
            sampling_radius -- search radius for next waypoints in seconds: e.g. 0.5 seconds ahead
            lateral_control_dict -- dictionary of arguments to setup the lateral PID controller
                                    {'K_P':, 'K_D':, 'K_I':, 'dt'}
            longitudinal_control_dict -- dictionary of arguments to setup the longitudinal PID controller
                                        {'K_P':, 'K_D':, 'K_I':, 'dt'}
        """
        self._vehicle = vehicle
        self._map = self._vehicle.get_world().get_map()

        self._dt = None
        self._target_speed = None
        self._sampling_radius = None                  # distance to search for next waypoints from current waypoint
        self._min_distance = None
        self._current_waypoint = None                 # waypoint at current location of the vehicle
        self._target_road_option = None
        self._next_waypoints = None
        self.target_waypoint = None
        self._vehicle_controller = None               # VehiclePIDController object
        self._global_plan = None                      #flag which sets whether we use a predifined trajectory queue or make one on the go
        # queue with tuples of (waypoint, RoadOption)
        self._waypoints_queue = deque(maxlen=20000)   #trajectory queue
        self._buffer_size = 5                         
        self._waypoint_buffer = deque(maxlen=self._buffer_size)   #first 5 elements from trajectory queue

        # initializing controller
        self._init_controller(opt_dict)
        # list of (carla.Waypoint, RoadOption); returned by trace_route function of Global Route Planner object
        self._current_plan=[]           

    def __del__(self):
        if self._vehicle:
            self._vehicle.destroy()
        # print("Destroying ego-vehicle!")

    def reset_vehicle(self):
        self._vehicle = None
        # print("Resetting ego-vehicle!")

    def _init_controller(self, opt_dict):
        """
        Controller initialization.
        :param opt_dict: dictionary of arguments.
        :return:
        """
        # default params
        self._dt = 1.0 / 20.0
        self._target_speed = 20.0  # Km/h
        self._sampling_radius = self._target_speed * 1 / 3.6  # 1 seconds horizon
        self._min_distance = self._sampling_radius * self.MIN_DISTANCE_PERCENTAGE
        args_lateral_dict = {
            'K_P': 2,
            'K_D': 0,
            'K_I': 1,
            'dt': self._dt}
        args_longitudinal_dict = {
            'K_P': 1.0,
            'K_D': 0,
            'K_I': 1,
            'dt': self._dt}

        # parameters overload
        if opt_dict:
            if 'dt' in opt_dict:
                self._dt = opt_dict['dt']
            if 'target_speed' in opt_dict:
                self._target_speed = opt_dict['target_speed']
            if 'sampling_radius' in opt_dict:
                self._sampling_radius = self._target_speed * \
                    opt_dict['sampling_radius'] / 3.6
            if 'lateral_control_dict' in opt_dict:
                args_lateral_dict = opt_dict['lateral_control_dict']
            if 'longitudinal_control_dict' in opt_dict:
                args_longitudinal_dict = opt_dict['longitudinal_control_dict']

        self._current_waypoint = self._map.get_waypoint(self._vehicle.get_location())
        self._vehicle_controller = VehiclePIDController(self._vehicle,
                                                       args_lateral=args_lateral_dict,
                                                       args_longitudinal=args_longitudinal_dict)

        self._global_plan = False

        # compute initial waypoints
        self._waypoints_queue.append((self._current_waypoint.next(self._sampling_radius)[0], RoadOption.LANEFOLLOW))

        self._target_road_option = RoadOption.LANEFOLLOW
        # fill waypoint trajectory queue
        self._compute_next_waypoints(k=200)      #add k waypoints to _waypoints_queue

    def set_speed(self, speed):
        """
        Request new target speed.
        :param speed: new target speed in Km/h
        :return:
        """
        self._target_speed = speed

    def _compute_next_waypoints(self, k=1):
        """
        Add new waypoints to the trajectory queue.
        :param k: how many waypoints to compute
        :return:
        """
        # check that we do not overflow the queue
        available_entries = self._waypoints_queue.maxlen - len(self._waypoints_queue)
        k = min(available_entries, k)

        for _ in range(k):
            last_waypoint = self._waypoints_queue[-1][0]
            next_waypoints = list(last_waypoint.next(self._sampling_radius))

            if len(next_waypoints) == 1:
                # only one option available ==> lanefollowing
                next_waypoint = next_waypoints[0]
                road_option = RoadOption.LANEFOLLOW
            else:
                # random choice between the possible options
                road_options_list = _retrieve_options(
                    next_waypoints, last_waypoint)
                road_option = random.choice(road_options_list)
                next_waypoint = next_waypoints[road_options_list.index(
                    road_option)]

            self._waypoints_queue.append((next_waypoint, road_option))

    def set_global_plan(self, current_plan):
        """
        Set trajectory queue to a predifined list of waypoints stored in current_plan
        :param current_plan: list of tuples (waypoint, RoadOption) 
        :return:
        """
        self._waypoints_queue.clear()              #remove all waypoints from the trajectory queue
        for elem in current_plan:
            self._waypoints_queue.append(elem)

        # print(current_plan[0][0])
        currplan = current_plan[0][0]              #first waypoint and RoadOption of our trajectory queue
        # print(currplan.transform.location.x)


#         sum = 0   #total distance over the trajectory of waypoints
#         for i in range(len(current_plan)-1):
#             sum = sum + self.distance_wp(current_plan[i+1][0], current_plan[i][0])


#         # print('The total distance over the road is: %0.2f' % sum)

        sum = self.total_distance(current_plan)


        self._target_road_option = RoadOption.LANEFOLLOW
        self._global_plan = True
        self._current_plan = current_plan

    def distance_wp(self,target,current):
        """
        Distance between target waypoint and current waypoint
        :param target : target waypoint
        :param current: current waypoint
        :return       : euclidean distance in metres between target and current waypoints
        """
        dx = target.transform.location.x - current.transform.location.x
        dy = target.transform.location.y - current.transform.location.y
        return math.sqrt(dx * dx + dy * dy)

    def total_distance(self,current_plan):
        """
        Calculates the total distance in metres over the trajectory of waypoints
        :param current_plan: list of tuples (waypoint, RoadOption)
        :return            : total distance in metres 
        """
        sum = 0
        for i in range(len(current_plan) - 1):
            sum = sum + self.distance_wp(current_plan[i + 1][0], current_plan[i][0])
        return sum

    def run_step(self, debug=True):
        """
        Execute one step of local planning which involves running the longitudinal and lateral PID controllers to
        follow the waypoints trajectory.
        :param debug: boolean flag to activate waypoints debugging
        :return d2wp: distance from current location of vehicle to next waypoint(first element of trajectory buufer)
              d2goal: distance from current location of vehicle to destination(last waypoint of trajectory queue)
                    : number of waypoints left on the trajectory queue
        """

        # not enough waypoints in the horizon? => add more!
        if not self._global_plan and len(self._waypoints_queue) < int(self._waypoints_queue.maxlen * 0.5):
            self._compute_next_waypoints(k=100)

        # if len(self._waypoints_queue) == 0:
        #     control = carla.VehicleControl()
        #     control.steer = 0.0
        #     control.throttle = 0.0
        #     control.brake = 1.0
        #     control.hand_brake = False
        #     control.manual_gear_shift = False
        #     d2goal = 0
        #     other_d2goal = 0
        #     return d2goal,other_d2goal,0,control
        if len(self._waypoints_queue) == 0:
            # control = carla.VehicleControl()
            # control.steer = 0.0
            # control.throttle = 0.0
            # control.brake = 1.0
            # control.hand_brake = False
            # control.manual_gear_shift = False
            d2wp = 8 # avoid divisions by zero
            d2goal = 10000 # avoid divisions by zero
            return d2wp, d2goal,0

        #   Buffering the waypoints
        if not self._waypoint_buffer:
            for i in range(self._buffer_size):
                if self._waypoints_queue:
                    #first element of trajectory queue is added to trajectory buffer
                    self._waypoint_buffer.append(self._waypoints_queue.popleft())   #poping removes the tuple from _waypoints_queue and sends it to _aypoint_buffer
                else:
                    break

        # current vehicle waypoint
        self._current_waypoint = self._map.get_waypoint(self._vehicle.get_location())
        # target waypoint
        self.target_waypoint, self._target_road_option = self._waypoint_buffer[0]
        # move using PID controllers
        # control = self._vehicle_controller.run_step(self._target_speed, self.target_waypoint)

        # purge the queue of obsolete waypoints
        vehicle_transform = self._vehicle.get_transform()
        max_index = -1

        for i, (waypoint, _) in enumerate(self._waypoint_buffer):
            if distance_vehicle(waypoint, vehicle_transform) < self._min_distance:
                max_index = i
        if max_index >= 0:
            for i in range(max_index + 1):
                self._waypoint_buffer.popleft()

        d2wp = distance_vehicle(self.target_waypoint,vehicle_transform)

        if debug:
            draw_waypoints(self._vehicle.get_world(), [self.target_waypoint], self._vehicle.get_location().z + 1.0)

        # other_d2goal = self.total_distance(self._current_plan)
        d2goal = self.total_distance(self._waypoints_queue)+d2wp

        # return d2goal, other_d2goal, len(self._waypoints_queue),control

        return d2wp,d2goal, len(self._waypoints_queue)


def _retrieve_options(list_waypoints, current_waypoint):
    """
    Compute the type of connection between the current active waypoint and the multiple waypoints present in
    list_waypoints. The result is encoded as a list of RoadOption enums.
    :param list_waypoints: list with the possible target waypoints in case of multiple options
    :param current_waypoint: current active waypoint
    :return: list of RoadOption enums representing the type of connection from the active waypoint to each
             candidate in list_waypoints
    """
    options = []
    for next_waypoint in list_waypoints:
        # this is needed because something we are linking to
        # the beggining of an intersection, therefore the
        # variation in angle is small
        next_next_waypoint = next_waypoint.next(3.0)[0]
        link = _compute_connection(current_waypoint, next_next_waypoint)
        options.append(link)

    return options


def _compute_connection(current_waypoint, next_waypoint):
    """
    Compute the type of topological connection between an active waypoint (current_waypoint) and a target waypoint
    (next_waypoint).
    :param current_waypoint: active waypoint
    :param next_waypoint: target waypoint
    :return: the type of topological connection encoded as a RoadOption enum:
             RoadOption.STRAIGHT
             RoadOption.LEFT
             RoadOption.RIGHT
    """
    n = next_waypoint.transform.rotation.yaw
    n = n % 360.0

    c = current_waypoint.transform.rotation.yaw
    c = c % 360.0

    diff_angle = (n - c) % 180.0
    if diff_angle < 1.0:
        return RoadOption.STRAIGHT
    elif diff_angle > 90.0:
        return RoadOption.LEFT
    else:
        return RoadOption.RIGHT

In [10]:
# ==============================================================================
# -- Get colors for debugging --------------------------------------------------
# ==============================================================================

red = carla.Color(255, 0, 0)
green = carla.Color(0, 255, 0)
blue = carla.Color(47, 210, 231)
cyan = carla.Color(0, 255, 255)
yellow = carla.Color(255, 255, 0)
orange = carla.Color(255, 162, 0)
white = carla.Color(255, 255, 255)

# ==============================================================================

@dataclass
class ACTIONS:
    forward_slow = 0
    forward_medium = 1
    forward_fast = 2
    left_slow = 3
    left_medium = 4
    left_fast = 5
    right_slow = 6
    right_medium = 7
    right_fast = 8
    brake_light = 9
    brake_medium = 10
    brake_full = 11
    no_action = 12

ACTION_CONTROL = {
    0: [0.3, 0, 0],
    1: [0.6, 0, 0],
    2: [1, 0, 0],
    3: [0.7, 0, -0.3],
    4: [0.7, 0, -0.6],
    5: [0.7, 0, -1],
    6: [0.7, 0, 0.3],
    7: [0.7, 0, 0.6],
    8: [0.7, 0, 1],
    9: [0, 0.3, 0],
    10: [0, 0.6, 0],
    11: [0, 1, 0],
    12: None,
}

ACTIONS_NAMES = {
    0: 'forward_slow',
    1: 'forward_medium',
    2: 'forward_fast',
    3: 'left_slow',
    4: 'left_medium',
    5: 'left_fast',
    6: 'right_slow',
    7: 'right_medium',
    8: 'right_fast',
    9: 'brake_light',
    10: 'brake_medium',
    11: 'brake_full',
    12: 'no_action',
}

In [11]:
class settings():

    # Carla environment settings
    IMG_WIDTH = 480
    IMG_HEIGHT = 270
    ACTIONS = ['forward_slow', 'forward_medium', 'forward_fast', 'left_slow', 'left_medium', 'left_fast', 'right_slow', 'right_medium', 'right_fast', 'brake_light', 'brake_medium', 'brake_full', 'no_action']
    SECONDS_PER_EPISODE = 10 # Original 1
    
    PREVIEW_CAMERA_RES = [[640, 400, -5, 0, 2.5], [1280, 800, -5, 0, 2.5]]  # Available resolutions from "above the car" preview camera [width, height, x, y, z], where x, y and z are related to car position
    
    
    
    

In [12]:
# Carla environment
class CarlaEnv(object):

    # How much steering to apply
    STEER_AMT = 1.0

    # Image dimensions (observation space)
    im_width = settings.IMG_WIDTH
    im_height = settings.IMG_HEIGHT

    # Action space size
    action_space_size = len(settings.ACTIONS)

    # ==============================================================================
    # -- Navigation functions -------------------------------------------------------
    # ==============================================================================

    def total_distance(self,current_plan):
        """
        Calculates the total distance in metres over the trajectory of waypoints
        :param current_plan: list of tuples (waypoint, RoadOption)
        :return            : total distance in metres 
        """
        sum = 0
        for i in range(len(current_plan) - 1):
            sum = sum + self.distance_wp(current_plan[i + 1][0], current_plan[i][0])
        return sum

    def distance_wp(self,target, current):
        """
        Distance between target waypoint and current waypoint
        :param target : target waypoint
        :param current: current waypoint
        :return       : euclidean distance in metres between target and current waypoints
        """
        dx = target.transform.location.x - current.transform.location.x
        dy = target.transform.location.y - current.transform.location.y
        return math.sqrt(dx * dx + dy * dy)

    def distance_goal(self, target, current):
        """
        :param target : carla.transform object
        :param current: carla.location object
        :return       : euclidean distance in metres
        """
        dx = target.location.x - current.x
        dy = target.location.y - current.y
        return math.sqrt(dx * dx + dy * dy)

    def distance_vehicle(self, waypoint, vehicle_transform):
        """
        Calcualtes distance between vehicle current location and target waypoint
        :param waypoint    : target waypoint
        :vehicle_transform : current transform(location + orientation) of vehicle
        :return            : euclidean distance in metres 
        """
        loc = vehicle_transform.location
        dx = waypoint.transform.location.x - loc.x
        dy = waypoint.transform.location.y - loc.y

        return math.sqrt(dx * dx + dy * dy)

    def draw_path(self, world, current_plan):
        """
        Draw lines in green through the list of waypoints which signify the route vehicle will follow
        : param world        : current carla.World of the vehicle
        : param current_plan : list of tuples (waypoint, RoadOption)
        """
        for i in range(len(current_plan) - 1):
            w1 = current_plan[i][0]
            w2 = current_plan[i + 1][0]
            self.world.debug.draw_line(w1.transform.location, w2.transform.location, thickness=0.5,
                                  color=green, life_time=120.0)
        self.draw_waypoint_info(world, current_plan[-1][0])

    def draw_waypoint_info(self, world, waypoint, lt=120):
        """
        Draw a point in red on the given waypoint
        """
        w_loc = waypoint.transform.location
        world.debug.draw_point(w_loc, 0.5, red, lt)

    # ==============================================================================

    def __init__(self, seconds_per_episode=None, playing=False):

        # Set a client and timeouts
        self.client = carla.Client("localhost", 2000)
        #self.client = carla.Client(*settings.CARLA_HOSTS[carla_instance][:2])
        self.client.set_timeout(5.0)

        # Get currently running world
        self.world = self.client.get_world()

        # Get list of actor blueprints
        self.blueprint_library = self.world.get_blueprint_library()

        # Get Tesla model 3 blueprint
        # self.model_3 = blueprint_library.filter('model3')[0]

        # Get a random blueprint.
        vehicles = self.world.get_blueprint_library().filter("vehicle.*")
        choices = [x for x in vehicles if int(x.get_attribute('number_of_wheels')) == 4]
        self.mycar = random.choice(choices)      #blueprint for randomly selected car
        # self.obs = random.choice(choices)

        # Sensors and helper lists
        self.collision_hist = []
        self.actor_list = []
        self.front_camera = None
        self.preview_camera = None
        self.frametimes = Queue()

        # Used to determine if Carla simulator is still working as it crashes quite often
        self.last_cam_update = time.time()

        # Updated by agents for statistics
        self.seconds_per_episode = seconds_per_episode

        # A flag indicating that we are just going to play
        self.playing = playing

        # Used with additional preview feature
        self.preview_camera_enabled = False # Originally False

        # Sets actually configured actions
        self.actions = [getattr(ACTIONS, action) for action in settings.ACTIONS]     #list of ints from 1 to 11

        # Set large initial distance to goal
        self.prev_d2goal = 10000
        # self.d2goal = 10000
        # self.d2wp = 8

        ##########################3
        self.map = self.world.get_map()
        self.d2goal = 10000
        # Initialize the route planner
        self.dao = GlobalRoutePlannerDAO(self.map, 2.0)  # Create a route for every 2m
        self.grp = GlobalRoutePlanner(self.dao)
        self.grp.setup()                                 #build the networkx graph
        ###################

    # Resets environment for new episode
    def reset(self):

        # ##########################3
        # self.map = self.world.get_map()
        # self.d2goal = 10000
        # # Initialize the route planner
        # self.dao = GlobalRoutePlannerDAO(self.map, 2.0)  # Create a route for every 2m
        # self.grp = GlobalRoutePlanner(self.dao)
        # self.grp.setup()                               #build the networkx graph
        # ###################

        # Car, sensors, etc. We create them every episode then destroy
        self.actor_list = []

        # When Carla breaks (stops working) or spawn point is already occupied, spawning a car throws an exception
        # We allow it to try for 3 seconds then forgive (will cause episode restart and in case of Carla broke inform
        # main thread that Carla simulator needs a restart)
        spawn_start = time.time()
        while True:
            try:
                # Get random spot from a list from predefined spots and try to spawn a car there
                # self.transform = random.choice(self.world.get_map().get_spawn_points())
                # self.vehicle = self.world.spawn_actor(self.model_3, self.transform)

                # First trace a route, then spawn the player
                spawn_points = self.map.get_spawn_points()
                while self.d2goal > 1000 or self.d2goal < 300:
                    pos_a = random.choice(spawn_points)
                    pos_b = random.choice(spawn_points)

                    a = pos_a.location
                    b = pos_b.location

                    self.current_plan = self.grp.trace_route(a, b)  # list of (carla.Waypoint, RoadOption) from origin (carla.Location) to destination (carla.Location)

                    self.d2goal = self.total_distance(self.current_plan)

                self.transform = pos_a      #carla.transform at which to spawn the vehicle
                self.vehicle = self.world.try_spawn_actor(self.mycar, self.transform)
                self.spectator = self.world.get_spectator() 
                self.spectator.set_transform(self.vehicle.get_transform())


                args_lateral_dict = {
                    'K_P': 2,  # 1
                    'K_D': 0.2,  # 0.02
                    'K_I': 1,
                    'dt': 1.0 / 20.0}
                target_speed = 50
                self._local_planner = ModifiedLocalPlanner(
                    self.vehicle, opt_dict={'target_speed': target_speed,
                                           'lateral_control_dict': args_lateral_dict})

                assert self.current_plan
                self._local_planner.set_global_plan(self.current_plan)
                self.current_plan = self.current_plan[0:len(self.current_plan) - 5][:]
                self.d2goal = self.total_distance(self.current_plan)
                # self.d2wp =  self.total_distance(self.current_plan[0:1][:])
                # self.goal = self.current_plan[-1][0]
                # self.start_point = self.current_plan[1][0]

                # Draw path for debugging
                self.draw_path(self.world, self.current_plan)

                break
            except:
                time.sleep(0.01)

            # If that can't be done in 3 seconds - forgive (and allow main process to handle for this problem)
            if time.time() > spawn_start + 10:
                raise Exception('Can\'t spawn a car')

        # Append actor to a list of spawned actors, we need to remove them later
        self.actor_list.append(self.vehicle)

        # Get the blueprint for the camera
        self.rgb_cam = self.blueprint_library.find('sensor.camera.rgb')
        # Set sensor resolution and field of view
        self.rgb_cam.set_attribute('image_size_x', f'{self.im_width}')
        self.rgb_cam.set_attribute('image_size_y', f'{self.im_height}')
        self.rgb_cam.set_attribute('fov', '110')

        # Set camera sensor relative to a car
        transform = carla.Transform(carla.Location(x=2.5, z=0.7))

        # Attach camera sensor to a car, so it will keep relative difference to it
        self.sensor = self.world.spawn_actor(self.rgb_cam, transform, attach_to=self.vehicle)

        # Register a callback called every time sensor sends a new data
        self.sensor.listen(self._process_img)

        # Add camera sensor to the list of actors
        self.actor_list.append(self.sensor)

        # Preview ("above the car") camera
        if self.preview_camera_enabled is not False:
            # Get the blueprint for the camera
            self.preview_cam = self.world.get_blueprint_library().find('sensor.camera.rgb')
            # Set sensor resolution and field of view
            self.preview_cam.set_attribute('image_size_x', f'{self.preview_camera_enabled[0]:0f}')
            self.preview_cam.set_attribute('image_size_y', f'{self.preview_camera_enabled[1]:0f}')
            self.preview_cam.set_attribute('fov', '110')

            # Set camera sensor relative to a car
            transform = carla.Transform(carla.Location(x=self.preview_camera_enabled[2], y=self.preview_camera_enabled[3], z=self.preview_camera_enabled[4]))

            # Attach camera sensor to a car, so it will keep relative difference to it
            self.preview_sensor = self.world.spawn_actor(self.preview_cam, transform, attach_to=self.vehicle)

            # Register a callback called every time sensor sends a new data
            self.preview_sensor.listen(self._process_preview_img)

            # Add camera sensor to the list of actors
            self.actor_list.append(self.preview_sensor)

        # Here's first workaround. If we do not apply any control it takes almost a second for car to start moving
        # after episode restart. That starts counting once we aplly control for a first time.
        # Workarounf here is to apply both throttle and brakes and disengage brakes once we are ready to start an episode.
        self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, brake=1.0))

        # And here's another workaround - it takes a bit over 3 seconds for Carla simulator to start accepting any
        # control commands. Above time adds to this one, but by those 2 tricks we start driving right when we start an episode
        # But that adds about 4 seconds of a pause time between episodes.
        time.sleep(4)

        # Collision history is a list callback is going to append to (we brake simulation on a collision)
        self.collision_hist = []

        # Get the blueprint of the sensor
        colsensor = self.world.get_blueprint_library().find('sensor.other.collision')

        # Create the collision sensor and attach ot to a car
        self.colsensor = self.world.spawn_actor(colsensor, carla.Transform(), attach_to=self.vehicle)

        # Register a callback called every time sensor sends a new data
        self.colsensor.listen(self._collision_data)

        # Add the collision sensor to the list of actors
        self.actor_list.append(self.colsensor)

        # Almost ready to start an episode, reset camera update variable
        self.last_cam_update = time.time()

        # Wait for a camera to send first image (important at the beginning of first episode)
        while self.front_camera is None or (self.preview_camera_enabled is not False and self.preview_camera is None):
            time.sleep(0.01)

        # Disengage brakes
        self.vehicle.apply_control(carla.VehicleControl(brake=0.0))

        # Remember a time of episode start (used to measure duration and set a terminal state)
        self.episode_start = time.time()

        # ## Add one vehicle on our specific route
        # obs_wp = self.current_plan[math.ceil(len(self.current_plan) / 3)][0]
        # car_actor = self.world.try_spawn_actor(self.obs, pos_b)
        # car_actor.set_transform(obs_wp.transform)
        # # car_actor.set_autopilot()
        # self.actor_list.append(car_actor)

        # # Get the blueprint of the sensor, I don't care about the poor other car, no col sensor
        # colsens = self.world.get_blueprint_library().find('sensor.other.collision')
        #
        # # Create the collision sensor and attach it to the car
        # colsensor = self.world.spawn_actor(colsens, carla.Transform(), attach_to=car_actor)
        #
        # # Register a callback called every time sensor sends a new data
        # colsensor.listen(self._collision_data)
        #
        # # Add the car and collision sensor to the list of car NPCs
        # self.actor_list.append(self.colsensor)

        # Return first observation space - current image from the camera sensor
        return [self.front_camera, 0]

    # Collision data callback handler
    def _collision_data(self, event):

        # What we collided with and what was the impulse
        collision_actor_id = event.other_actor.type_id
        collision_impulse = math.sqrt(event.normal_impulse.x ** 2 + event.normal_impulse.y ** 2 + event.normal_impulse.z ** 2)

        # Filter collisions
        for actor_id, impulse in settings.COLLISION_FILTER:
            if actor_id in collision_actor_id and (impulse == -1 or collision_impulse <= impulse):
                return

        # Add collision
        self.collision_hist.append(event)

    # Camera sensor data callback handler
    def _process_img(self, image):

        # Get image, reshape and drop alpha channel
        image = np.array(image.raw_data)
        image = image.reshape((self.im_height, self.im_width, 4))
        image = image[:, :, :3]

        # Set as a current frame in environment
        self.front_camera = image

        # Measure frametime using a time of last camera update (displayed as Carla FPS)
        if self.playing:
            self.frametimes.append(time.time() - self.last_cam_update)
        else:
            self.frametimes.put_nowait(time.time() - self.last_cam_update)
        self.last_cam_update = time.time()

    # Preview camera sensor data callback handler
    def _process_preview_img(self, image):

        # If camera is disabled - do not process images
        if self.preview_camera_enabled is False:
            return

        # Get image, reshape and drop alpha channel
        image = np.array(image.raw_data)
        try:
            image = image.reshape((int(self.preview_camera_enabled[1]), int(self.preview_camera_enabled[0]), 4))
        except:
            return
        image = image[:, :, :3]

        # Set as a current frame in environment
        self.preview_camera = image
        
        
    # Steps environment
    def step(self, action):

        # Monitor if carla stopped sending images for longer than a second. If yes - it broke
        if time.time() > self.last_cam_update + 5:
            raise Exception('Missing updates from Carla')

        # Apply control to the vehicle based on an action
        if self.actions[action] != ACTIONS.no_action:
            self.vehicle.apply_control(carla.VehicleControl(throttle=ACTION_CONTROL[self.actions[action]][0], 
                                                            steer=ACTION_CONTROL[self.actions[action]][2]*self.STEER_AMT, 
                                                            brake=ACTION_CONTROL[self.actions[action]][1]))

        # Calculate speed in km/h from car's velocity (3D vector)
        v = self.vehicle.get_velocity()
        kmh = 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)

        d2wp, self.d2goal, wp_in_line = self._local_planner.run_step(debug=False)
        d2wp = 10
        d2goal = 100

        done = False

        eps = 5          #threshold in metres for measuring if destination is reached
        d0 = 8           #threshold for measuring nearness to next waypoint
        kmh0 = 50        #threshold for minimum driving speed

        # If car collided - end and episode and send back a penalty
        if len(self.collision_hist) != 0:
            done = True
            reward = -1

        elif d2goal > eps:
            reward = 1 - d2goal / self.prev_d2goal - d2wp / d0 + kmh / kmh0

        elif d2goal <= eps:
            done = True
            reward = 100 # original 1

        # If episode duration limit reached - send back a terminal state
        if not self.playing and self.episode_start + self.seconds_per_episode.value < time.time():
            done = True

        # Weights rewards (not for terminal state)
        if not self.playing and settings.WEIGHT_REWARDS_WITH_EPISODE_PROGRESS and not done:
            reward *= (time.time() - self.episode_start) / self.seconds_per_episode.value

        self.prev_d2goal = d2goal

        return [self.front_camera, kmh, d2wp, d2goal], reward, done, None

    # Destroys all agents created from last .reset() call
    def destroy_agents(self):

        for actor in self.actor_list:

            # If it has a callback attached, remove it first
            if hasattr(actor, 'is_listening') and actor.is_listening:
                actor.stop()

            # If it's still alive - destroy it
            if actor.is_alive:
                actor.destroy()

        self.actor_list = []


# operating_system = operating_system()


# # Returns binary
# def get_binary():
#     return 'CarlaUE4.exe' if operating_system == 'windows' else 'CarlaUE4.sh'


# # Returns exec command
# def get_exec_command():
#     binary = get_binary()
#     exec_command = binary if operating_system == 'windows' else ('./' + binary)

#     return binary, exec_command


# # tries to close, and if that does not work to kill all carla processes
# def kill_processes():

#     binary = get_binary()

#     # Iterate processes and terminate carla ones
#     for process in psutil.process_iter():
#         if process.name().lower().startswith(binary.split('.')[0].lower()):
#             try:
#                 process.terminate()
#             except:
#                 pass

#     # Check if any are still alive, create a list
#     still_alive = []
#     for process in psutil.process_iter():
#         if process.name().lower().startswith(binary.split('.')[0].lower()):
#             still_alive.append(process)

#     # Kill process and wait until it's being killed
#     if len(still_alive):
#         for process in still_alive:
#             try:
#                 process.kill()
#             except:
#                 pass
#         psutil.wait_procs(still_alive)


# # Starts Carla simulator
# def start(playing=False):
#     # Kill Carla processes if there are any and start simulator
#     if settings.CARLA_HOSTS_TYPE == 'local':
#         print('Starting Carla...')
#         kill_processes()
#         for process_no in range(1 if playing else settings.CARLA_HOSTS_NO):
#             # subprocess.Popen(get_exec_command()[1] + f' -carla-rpc-port={settings.CARLA_HOSTS[process_no][1]}', cwd=settings.CARLA_PATH, shell=True)
#             subprocess.Popen('SDL_VIDEODRIVER=offscreen ' + get_exec_command()[1] + f' -carla-rpc-port={settings.CARLA_HOSTS[process_no][1]}', cwd=settings.CARLA_PATH, shell=True)
#             time.sleep(2)

#     # Else just wait for it to be ready
#     else:
#         print('Waiting for Carla...')

#     # Wait for Carla Simulator to be ready
#     for process_no in range(1 if playing else settings.CARLA_HOSTS_NO):
#         while True:
#             try:
#                 client = carla.Client(*settings.CARLA_HOSTS[process_no][:2])
#                 # settings.no_rendering_mode = True
#                 map_name = client.get_world().get_map().name
#                 if len(settings.CARLA_HOSTS[process_no]) == 2 or not settings.CARLA_HOSTS[process_no][2]:
#                     break
#                 if isinstance(settings.CARLA_HOSTS[process_no][2], int):
#                     map_choice = random.choice([map.split('/')[-1] for map in client.get_available_maps()])
#                 else:
#                     map_choice = settings.CARLA_HOSTS[process_no][2]
#                 if map_name != map_choice:
#                     carla.Client(*settings.CARLA_HOSTS[process_no][:2]).load_world(map_choice)
#                     while True:
#                         try:
#                             while carla.Client(*settings.CARLA_HOSTS[process_no][:2]).get_world().get_map().name != map_choice:
#                                 time.sleep(0.1)
#                             break
#                         except:
#                             pass
#                 break
#             except Exception as e:
#                 #print(str(e))
#                 time.sleep(0.1)


# # Retarts Carla simulator
# def restart(playing=False):
#     # Kill Carla processes if there are any and start simulator
#     if settings.CARLA_HOSTS_TYPE == 'local':
#         for process_no in range(1 if playing else settings.CARLA_HOSTS_NO):
#             # subprocess.Popen(get_exec_command()[1] + f' -carla-rpc-port={settings.CARLA_HOSTS[process_no][1]}', cwd=settings.CARLA_PATH, shell=True)
#             subprocess.Popen('SDL_VIDEODRIVER=offscreen ' + get_exec_command()[1] + f' -carla-rpc-port={settings.CARLA_HOSTS[process_no][1]}',
#                              cwd=settings.CARLA_PATH, shell=True)
#             time.sleep(2)

#     # Wait for Carla Simulator to be ready
#     for process_no in range(1 if playing else settings.CARLA_HOSTS_NO):
#         retries = 0
#         while True:
#             try:
#                 client = carla.Client(*settings.CARLA_HOSTS[process_no][:2])
#                 map_name = client.get_world().get_map().name
#                 if len(settings.CARLA_HOSTS[process_no]) == 2 or not settings.CARLA_HOSTS[process_no][2]:
#                     break
#                 if isinstance(settings.CARLA_HOSTS[process_no][2], int):
#                     map_choice = random.choice([map.split('/')[-1] for map in client.get_available_maps()])
#                 else:
#                     map_choice = settings.CARLA_HOSTS[process_no][2]
#                 if map_name != map_choice:
#                     carla.Client(*settings.CARLA_HOSTS[process_no][:2]).load_world(map_choice)
#                     while True:
#                         try:
#                             while carla.Client(*settings.CARLA_HOSTS[process_no][:2]).get_world().get_map().name != map_choice:
#                                 time.sleep(0.1)
#                                 retries += 1
#                                 if retries >= 60:
#                                     raise Exception('Couldn\'t change map [1]')
#                             break
#                         except Exception as e:
#                             time.sleep(0.1)
#                         retries += 1
#                         if retries >= 60:
#                             raise Exception('Couldn\'t change map [2]')

#                 break
#             except Exception as e:
#                 #print(str(e))
#                 time.sleep(0.1)

#             retries += 1
#             if retries >= 60:
#                 break


# # Parts of weather control code and npc car spawn code are copied from dynamic_weather.py and spawn_npc.py from examples, then modified
# def clamp(value, minimum=0.0, maximum=100.0):
#     return max(minimum, min(value, maximum))


# class Sun(object):
#     def __init__(self, azimuth, altitude):
#         self.azimuth = azimuth
#         self.altitude = altitude
#         self._t = 0.0

#     def tick(self, delta_seconds):
#         self._t += 0.008 * delta_seconds
#         self._t %= 2.0 * math.pi
#         self.azimuth += 0.25 * delta_seconds
#         self.azimuth %= 360.0
#         self.altitude = 35.0 * (math.sin(self._t) + 1.0)


# class Storm(object):
#     def __init__(self, precipitation):
#         self._t = precipitation if precipitation > 0.0 else -50.0
#         self._increasing = True
#         self.clouds = 0.0
#         self.rain = 0.0
#         self.puddles = 0.0
#         self.wind = 0.0

#     def tick(self, delta_seconds):
#         delta = (1.3 if self._increasing else -1.3) * delta_seconds
#         self._t = clamp(delta + self._t, -250.0, 100.0)
#         self.clouds = clamp(self._t + 40.0, 0.0, 90.0)
#         self.rain = clamp(self._t, 0.0, 80.0)
#         delay = -10.0 if self._increasing else 90.0
#         self.puddles = clamp(self._t + delay, 0.0, 75.0)
#         self.wind = clamp(self._t - delay, 0.0, 80.0)
#         if self._t == -250.0:
#             self._increasing = True
#         if self._t == 100.0:
#             self._increasing = False


# class Weather(object):
#     def __init__(self, weather):
#         self.weather = weather
#         self.sun = Sun(weather.sun_azimuth_angle, weather.sun_altitude_angle)
#         self.storm = Storm(weather.precipitation)

#     def set_new_weather(self, weather):
#         self.weather = weather

#     def tick(self, delta_seconds):
#         delta_seconds += random.uniform(-0.1, 0.1)
#         self.sun.tick(delta_seconds)
#         self.storm.tick(delta_seconds)
#         self.weather.cloudyness = self.storm.clouds
#         self.weather.precipitation = self.storm.rain
#         self.weather.precipitation_deposits = self.storm.puddles
#         self.weather.wind_intensity = self.storm.wind
#         self.weather.sun_azimuth_angle = self.sun.azimuth
#         self.weather.sun_altitude_angle = self.sun.altitude


# # Carla settings states
# @dataclass
# class CARLA_SETTINGS_STATE:
#     starting = 0
#     working = 1
#     restarting = 2
#     finished = 3
#     error = 4


# # Carla settings state messages
# CARLA_SETTINGS_STATE_MESSAGE = {
#     0: 'STARTING',
#     1: 'WORKING',
#     2: 'RESTARTING',
#     3: 'FINISHED',
#     4: 'ERROR',
# }

# # Carla settings class
# class CarlaEnvSettings:

#     def __init__(self, process_no, agent_pauses, stop=None, car_npcs=[0, 0], stats=[0., 0., 0., 0., 0., 0.]):

#         # Speed factor changes how fast weather should change
#         self.speed_factor = 1.0

#         # Process number (Carla instance to use)
#         self.process_no = process_no

#         # Weather and NPC variables
#         self.weather = None
#         self.spawned_car_npcs = {}

#         # Set stats (for Tensorboard)
#         self.stats = stats

#         # Set externally to restarts settings
#         self.restart = False

#         # Controls number of NPCs and reset interval
#         self.car_npcs = car_npcs

#         # State for stats
#         self.state = CARLA_SETTINGS_STATE.starting

#         # External stop object (used to "know" when to exit
#         self.stop = stop

#         # We want to track NPC collisions so we can remove and spawn new ones
#         # Collisions are really not rare when using built-in autopilot
#         self.collisions = Queue()

#         # Name of current world
#         self.world_name = None

#         # Controls world reloads
#         self.reload_world_every = None if len(settings.CARLA_HOSTS[process_no]) == 2 or not settings.CARLA_HOSTS[process_no][2] or not isinstance(settings.CARLA_HOSTS[process_no][2], int) else (settings.CARLA_HOSTS[process_no][2] + random.uniform(-settings.CARLA_HOSTS[process_no][2]/10, settings.CARLA_HOSTS[process_no][2]/10))*60
#         self.next_world_reload = None if self.reload_world_every is None else time.time() + self.reload_world_every

#         # List of communications objects allowing Carla to pause agents (on changes like world change)
#         self.agent_pauses = agent_pauses

#         # Initialize map and route
#         ##########################3
#         # self.map = self.world.get_map()
#         # self.d2goal = 10000
#         # # Initialize the route planner
#         # self.dao = GlobalRoutePlannerDAO(self.map, 2.0)  # Create a route for every 2m
#         # self.grp = GlobalRoutePlanner(self.dao)
#         # self.grp.setup()
#         #
#         # spawn_points = self.map.get_spawn_points()
#         # while self.d2goal > 150 or self.d2goal < 50:
#         #     pos_a = random.choice(spawn_points)
#         #     pos_b = random.choice(spawn_points)
#         #
#         #     a = pos_a.location
#         #     b = pos_b.location
#         #
#         #     self.current_plan = self.grp.trace_route(a, b)
#         #
#         #     self.d2goal = self.total_distance(self.current_plan)
#         #
#         # self.transform = pos_a
#         ###################

#     # Collect NPC collision data
#     def _collision_data(self, collision):
#         self.collisions.put(collision)

#     # Destroys given car NPC
#     def _destroy_car_npc(self, car_npc):

#         # First check if NPC is still alive
#         if car_npc in self.spawned_car_npcs:

#             # Iterate all agents (currently car itself and collision sensor)
#             for actor in self.spawned_car_npcs[car_npc]:

#                 # If actor has any callback attached - stop it
#                 if hasattr(actor, 'is_listening') and actor.is_listening:
#                     actor.stop()

#                 # And if is still alive - destroy it
#                 if actor.is_alive:
#                     actor.destroy()

#             # Remove from car NPCs' list
#             del self.spawned_car_npcs[car_npc]

#     def clean_carnpcs(self):

#         # If there were any NPC cars - remove attached callbacks from it's agents
#         for car_npc in self.spawned_car_npcs.keys():
#             for actor in self.spawned_car_npcs[car_npc]:
#                 if hasattr(actor, 'is_listening') and actor.is_listening:
#                     actor.stop()
#                     # avoid sensor stream error
#                     # actor.destroy()

#         # Reset NPC car list
#         self.spawned_car_npcs = {}

#     # Main method, being run in a thread
#     def update_settings_in_loop(self):

#         # Reset world name
#         self.world_name = None

#         # Reset weather object
#         self.weather = None

#         # Run infinitively
#         while True:

#             # Release agent pause locks, if there are any
#             for agent_pause in self.agent_pauses:
#                 agent_pause.value = 0

#             # Carla might break, make sure we can handle for that
#             try:

#                 # If stop flag - exit
#                 if self.stop is not None and self.stop.value == STOP.stopping:
#                     self.state = CARLA_SETTINGS_STATE.finished
#                     return

#                 # If restart flag is being set - wait
#                 if self.restart:
#                     self.state = CARLA_SETTINGS_STATE.restarting
#                     time.sleep(0.1)
#                     continue

#                 # Clean car npcs
#                 self.clean_carnpcs()

#                 # Connect to Carla, get worls and map
#                 self.client = carla.Client(*settings.CARLA_HOSTS[self.process_no][:2])
#                 self.client.set_timeout(2.0)
#                 self.world = self.client.get_world()
#                 self.map = self.world.get_map()
#                 self.world_name = self.map.name

#                 # Create weather object or update it if exists
#                 if self.weather is None:
#                     self.weather = Weather(self.world.get_weather())
#                 else:
#                     self.weather.set_new_weather(self.world.get_weather())

#                 # Get car blueprints and filter them
#                 self.car_blueprints = self.world.get_blueprint_library().filter('vehicle.*')
#                 self.car_blueprints = [x for x in self.car_blueprints if int(x.get_attribute('number_of_wheels')) == 4]
#                 self.car_blueprints = [x for x in self.car_blueprints if not x.id.endswith('isetta')]
#                 self.car_blueprints = [x for x in self.car_blueprints if not x.id.endswith('carlacola')]

#                 # Get a list of all possible spawn points
#                 self.spawn_points = self.map.get_spawn_points()

#                 # Get collision sensor blueprint
#                 self.collision_sensor = self.world.get_blueprint_library().find('sensor.other.collision')

#                 # Used to know when to reset next NPC car
#                 car_despawn_tick = 0

#                 # Set state to working
#                 self.state = CARLA_SETTINGS_STATE.working

#             # In case of error, report it, wait a second and try again
#             except Exception as e:
#                 self.state = CARLA_SETTINGS_STATE.error
#                 time.sleep(1)
#                 continue

#             # Steps all settings
#             while True:

#                 # Used to measure sleep time at the loop end
#                 step_start = time.time()

#                 # If stop flag - exit
#                 if self.stop is not None and self.stop.value == STOP.stopping:
#                     self.state = CARLA_SETTINGS_STATE.finished
#                     return

#                 # Is restart flag is being set, break inner loop
#                 if self.restart:
#                     break

#                 # Carla might break, make sure we can handle for that
#                 try:

#                     # World reload
#                     if self.next_world_reload and time.time() > self.next_world_reload:

#                         # Set restart flag
#                         self.state = CARLA_SETTINGS_STATE.restarting
#                         # Clean car npcs
#                         self.clean_carnpcs()

#                         # Set pause lock flag
#                         for agent_pause in self.agent_pauses:
#                             agent_pause.value = 1

#                         # Wait for agents to stop playing and acknowledge
#                         for agent_pause in self.agent_pauses:
#                             while agent_pause.value != 2:
#                                 time.sleep(0.1)

#                         # Get random map and load it
#                         map_choice = random.choice(list({map.split('/')[-1] for map in self.client.get_available_maps()} - {self.client.get_world().get_map().name}))
#                         self.client.load_world(map_choice)

#                         # Wait for world to be fully loaded
#                         retries = 0
#                         while self.client.get_world().get_map().name != map_choice:
#                             retries += 1
#                             if retries >= 600:
#                                 raise Exception('Timeout when waiting for new map to be fully loaded')
#                             time.sleep(0.1)

#                         # Inform agents that they can start playing
#                         for agent_pause in self.agent_pauses:
#                             agent_pause.value = 3

#                         # Wait for agents to start playing
#                         for agent_pause in self.agent_pauses:
#                             retries = 0
#                             while agent_pause.value != 0:
#                                 retries += 1
#                                 if retries >= 600:
#                                     break
#                             time.sleep(0.1)

#                         self.next_world_reload += self.reload_world_every

#                         break

#                     # Handle all registered collisions
#                     while not self.collisions.empty():

#                         # Gets first collision from the queue
#                         collision = self.collisions.get()

#                         # Gets car NPC's id and destroys it
#                         car_npc = collision.actor.id
#                         self._destroy_car_npc(car_npc)

#                     # Count tick
#                     car_despawn_tick += 1

#                     # Carla autopilot might cause cars to stop in the middle of intersections blocking whole traffic
#                     # On some intersections there might be only one car moving
#                     # We want to check for cars stopped at intersections and remove them
#                     # Without that most of the cars can be waiting around 2 intersections
#                     for car_npc in self.spawned_car_npcs.copy():

#                         # First check if car is moving
#                         # It;s a simple check, not proper velocity calculation
#                         velocity = self.spawned_car_npcs[car_npc][0].get_velocity()
#                         simple_speed = velocity.x + velocity.y + velocity.z

#                         # If car is moving, continue loop
#                         if simple_speed > 0.1 or simple_speed < -0.1:
#                             continue

#                         # Next get current location of the car, then a waypoint then check if it's intersection
#                         location = self.spawned_car_npcs[car_npc][0].get_location()
#                         waypoint = self.map.get_waypoint(location)
#                         if not waypoint.is_intersection:
#                             continue

#                         # Car is not moving, it's intersection - destroy a car
#                         self._destroy_car_npc(car_npc)

#                     # If we reached despawn tick, remove oldest NPC
#                     # The reason we want to do that is to rotate cars around the map
#                     if car_despawn_tick >= self.car_npcs[1] and len(self.spawned_car_npcs):

#                         # Get id of the first car on a list and destroy it
#                         car_npc = list(self.spawned_car_npcs.keys())[0]
#                         self._destroy_car_npc(car_npc)
#                         car_despawn_tick = 0

#                     # If there is less number of car NPCs then desired amount - spawn remaining ones
#                     # but up to 10 at the time
#                     if len(self.spawned_car_npcs) < self.car_npcs[0]:

#                         # How many cars to spawn (up to 10)
#                         cars_to_spawn = min(10, self.car_npcs[0] - len(self.spawned_car_npcs))

#                         # Sometimes we cannot spawn a car
#                         # It might be because spawn point is being occupied or because Carla broke
#                         # We count errors and break on 5
#                         retries = 0

#                         # Iterate over number of cars to spawn
#                         for _ in range(cars_to_spawn):

#                             # Break if too many errors
#                             if retries >= 5:
#                                 break

#                             # Get random car blueprint and randomize color and enable autopilot
#                             car_blueprint = random.choice(self.car_blueprints)
#                             if car_blueprint.has_attribute('color'):
#                                 color = random.choice(car_blueprint.get_attribute('color').recommended_values)
#                                 car_blueprint.set_attribute('color', color)
#                             car_blueprint.set_attribute('role_name', 'autopilot')

#                             # Try to spawn a car
#                             # spawn_point = random.choice(self.spawn_points)
#                             # car_actor = self.world.try_spawn_actor(car_blueprint, spawn_point)
#                             # new_spawn_point = CarlaEnv.current_plan[math.ceil(len(self.current_plan) / 2)][0]
#                             # car_actor.set_transform(new_spawn_point)
#                             # car_actor.set_autopilot()

#                             # Try to spawn a car with more cars
#                             for _ in range(5):
#                                 try:
#                                     # Get random spot from a list from predefined spots and try to spawn a car there
#                                     spawn_point = random.choice(self.spawn_points)
#                                     car_actor = self.world.spawn_actor(car_blueprint, spawn_point)
#                                     # new_spawn_point = CarlaEnv.current_plan[math.ceil(len(self.current_plan) / 2)][0]
#                                     # car_actor.set_transform(new_spawn_point)
#                                     car_actor.set_autopilot()
#                                     break
#                                 except:
#                                     retries += 1
#                                     time.sleep(0.1)
#                                     continue

#                             # Create the collision sensor and attach it to the car
#                             colsensor = self.world.spawn_actor(self.collision_sensor, carla.Transform(), attach_to=car_actor)

#                             # Register a callback called every time sensor sends a new data
#                             colsensor.listen(self._collision_data)

#                             # Add the car and collision sensor to the list of car NPCs
#                             self.spawned_car_npcs[car_actor.id] = [car_actor, colsensor]

#                     # Tick a weather and set it in Carla
#                     self.weather.tick(self.speed_factor)
#                     self.world.set_weather(self.weather.weather)

#                     # Set stats for tensorboard
#                     self.stats[0] = len(self.spawned_car_npcs)
#                     self.stats[1] = self.weather.sun.azimuth
#                     self.stats[2] = self.weather.sun.altitude
#                     self.stats[3] = self.weather.storm.clouds
#                     self.stats[4] = self.weather.storm.wind
#                     self.stats[5] = self.weather.storm.rain

#                     # In case of state being some other one report that everything is working
#                     self.state = CARLA_SETTINGS_STATE.working

#                     # Calculate how long to sleep and sleep
#                     sleep_time = self.speed_factor - time.time() + step_start
#                     if sleep_time > 0:
#                         time.sleep(sleep_time)

#                 # In case of error, report it (reset flag set externally might break this loop only)
#                 except Exception as e:
#                     #print(str(e))
#                     self.state = CARLA_SETTINGS_STATE.error

In [13]:
env = CarlaEnv(settings.SECONDS_PER_EPISODE)

In [14]:
_ = env.reset()

In [15]:
env.destroy_agents()

In [ ]:
actor_list = []

In [ ]:
spawn_points = env.map.get_spawn_points()
while env.d2goal > 150 or env.d2goal < 50:
    pos_a = random.choice(spawn_points)
    pos_b = random.choice(spawn_points)

    a = pos_a.location
    b = pos_b.location

    env.current_plan = env.grp.trace_route(a, b)  # list of (carla.Waypoint, RoadOption) from origin (carla.Location) to destination (carla.Location)

    env.d2goal = env.total_distance(env.current_plan)

env.transform = pos_a      #carla.transform at which to spawn the vehicle
env.vehicle = env.world.spawn_actor(env.mycar, env.transform)
actor_list.append(env.vehicle)

env.spectator = env.world.get_spectator() 
env.spectator.set_transform(env.vehicle.get_transform())

args_lateral_dict = {
    'K_P': 2,  # 1
    'K_D': 0.2,  # 0.02
    'K_I': 1,
    'dt': 1.0 / 20.0}
target_speed = 50
env._local_planner = ModifiedLocalPlanner(env.vehicle, opt_dict={'target_speed': target_speed, 'lateral_control_dict': args_lateral_dict})






In [ ]:
assert env.current_plan
env._local_planner.set_global_plan(env.current_plan)
env.current_plan = env.current_plan[0:len(env.current_plan) - 5][:]
env.d2goal = env.total_distance(env.current_plan)
# self.d2wp =  self.total_distance(self.current_plan[0:1][:])
# self.goal = self.current_plan[-1][0]
# self.start_point = self.current_plan[1][0]

In [ ]:
# Draw path for debugging
env.draw_path(env.world, env.current_plan)

In [ ]:
type(env.vehicle)

In [ ]:
for actor in env.actor_list:
    actor.destroy()

In [ ]:
actor_list

In [ ]:
env.current_plan

In [ ]:
DAO = GlobalRoutePlannerDAO(wmap)
GRP = GlobalRoutePlanner(DAO)
GRP.setup()


In [ ]:
print(len(GRP._id_map), len(GRP._road_id_to_edge))
len(GRP._graph.edges())

In [ ]:
topology = GRP._dao.get_topology()
graph, id_map, road_id_to_edge = GRP._build_graph()

print(len(id_map), len(road_id_to_edge))
len(graph.edges())

In [ ]:
GRP._find_loose_ends()

print(len(id_map), len(road_id_to_edge))
len(graph.edges())

In [ ]:
GRP._lane_change_link()

print(len(id_map), len(road_id_to_edge))
len(graph.edges())

In [ ]:
l1 = np.array((2, 3, 5))
l2 = np.array((3, 4, 7))

In [ ]:
np.linalg.norm(l1-l2)

In [ ]:
math.radians(5)

In [ ]:
draw_waypoints(world, topology[3]['path'])

In [ ]:
segment = topology[0]

In [ ]:
import networkx as nx
graph = nx.DiGraph()

In [ ]:
entry_xyz, exit_xyz = segment['entryxyz'], segment['exitxyz']
path = segment['path']
entry_wp, exit_wp = segment['entry'], segment['exit']

In [ ]:
for vertex in [entry_xyz, exit_xyz]:
    print(type(vertex))
    print(vertex)

In [ ]:
graph.add_node(0, vertex=entry_xyz)
graph.add_node(1, vertex=exit_xyz)

In [ ]:
nx.draw(graph, with_labels=True)

In [ ]:
graph.nodes()

In [ ]:
graph.edges()

In [ ]:
ACTION_CONTROL = {
    0: [0.3, 0, 0],
    1: [0.6, 0, 0],
    2: [1, 0, 0],
    3: [0.7, 0, -0.3],
    4: [0.7, 0, -0.6],
    5: [0.7, 0, -1],
    6: [0.7, 0, 0.3],
    7: [0.7, 0, 0.6],
    8: [0.7, 0, 1],
    9: [0, 0.3, 0],
    10: [0, 0.6, 0],
    11: [0, 1, 0],
    12: None,
}

In [ ]:
type(ACTION_CONTROL)

In [ ]:
from dataclasses import dataclass

@dataclass
class ACTIONS:
    forward_slow = 0
    forward_medium = 1
    forward_fast = 2
    left_slow = 3
    left_medium = 4
    left_fast = 5
    right_slow = 6
    right_medium = 7
    right_fast = 8
    brake_light = 9
    brake_medium = 10
    brake_full = 11
    no_action = 12
    
actions = ACTIONS()


In [ ]:
type(actions.forward_slow)

In [ ]:
actions.forward_slow

In [ ]:
current_plan = [(1, "one"), (53, "two"), (23, "one"), (44, "three")]

In [ ]:
print(current_plan[0][0])
print(current_plan[0][1])
print(current_plan[1][0])
print(current_plan[1][1])
print(current_plan[2][0])
print(current_plan[2][1])
print(current_plan[3][0])
print(current_plan[3][1])

In [ ]:
from queue import Queue
from collections import deque

In [ ]:
waypoints_queue = deque(maxlen=20000)   #trajectory queue
waypoints_queue.append((1, "one"))
waypoints_queue.append((2, "two"))
# buffer_size = 5                         
# waypoint_buffer = deque(maxlen=self._buffer_size)   #first 5 elements from trajectory queue

waypoints_queue


In [ ]:
waypoints_queue.pop()

In [ ]:
waypoints_queue

In [ ]:
waypoints_queue.popleft()

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class ACTIONS:
    forward_slow = 0
    forward_medium = 1
    forward_fast = 2
    left_slow = 3
    left_medium = 4
    left_fast = 5
    right_slow = 6
    right_medium = 7
    right_fast = 8
    brake_light = 9
    brake_medium = 10
    brake_full = 11
    no_action = 12

In [ ]:
ACTION = ['forward_slow', 'forward_medium', 'forward_fast', 'left_slow', 'left_medium', 'left_fast', 'right_slow', 'right_medium', 'right_fast', 'brake_light', 'brake_medium', 'brake_full']

In [ ]:
[getattr(ACTIONS, action) for action in ACTION]